In [1]:
!pip install numpy matplotlib
!pip install ipython
!pip install qiskit


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [2]:
from quantum_gate_package import I, X, Y, Z, P0, P1, H,Y,S,T,rotation_gate,CNOT

Create a random pure state \(\rho\)  
Compute both forms:

\[
A = (1-p)\rho + p\frac{I}{2}
\]

\[
B = \left(1-\frac{3p}{4}\right)\rho + \frac{p}{4}\left(X\rho X + Y\rho Y + Z\rho Z\right)
\]

Print the norm \( \|A - B\| \) (should be \(\approx 0\))  
Then simulate random Pauli errors (Monte Carlo) to show the same result


Depolarizing channel discretization

We verify:

\[
(1-p)\rho + p\frac{\mathbb{I}}{2}
=
\left(1-\frac{3p}{4}\right)\rho
+
\frac{p}{4}\left(X\rho X + Y\rho Y + Z\rho Z\right).
\]

So depolarizing noise becomes a mixture of Pauli errors.


TODO:
Pick random pure state rho
Compute A = (1-p)rho + p*I/2
Compute B = (1-3p/4)rho + (p/4)(XrhoX + YrhoY + ZrhoZ)
Show ||A-B|| ~ 0


In [3]:
import numpy as np

def random_pure_state(seed=0):
    rng = np.random.default_rng(seed)
    v = rng.normal(size=2) + 1j*rng.normal(size=2)
    v = v / np.linalg.norm(v)
    return v

def rho_from_state(psi):
    return np.outer(psi, np.conjugate(psi))

def depolarizing_A(rho, p):
    return (1-p)*rho + p*(I/2)

def depolarizing_B(rho, p):
    return (1 - 3*p/4)*rho + (p/4)*(X@rho@X + Y@rho@Y + Z@rho@Z)

# ---- Example test
p = 0.37
psi = random_pure_state(seed=3)
rho = rho_from_state(psi)

A = depolarizing_A(rho, p)
B = depolarizing_B(rho, p)

print("p =", p)
print("||A - B|| =", np.linalg.norm(A - B))
print("Should be ~ 0 (numerical precision)")


p = 0.37
||A - B|| = 9.615595855602389e-17
Should be ~ 0 (numerical precision)


Code 2: Monte Carlo simulation example (Pauli-mixture = depolarizing)

This simulates the depolarizing channel by random Pauli errors:

- with probability \(1-\frac{3p}{4}\): do nothing  
- with probability \(\frac{p}{4}\): apply \(X\), \(Y\), or \(Z\)

Then we compare the Monte Carlo averaged output density matrix to the direct depolarizing formula \(A\).

In [4]:
def apply_pauli_channel_monte_carlo(rho, p, shots=20000, seed=0):
    rng = np.random.default_rng(seed)

    # Probabilities
    pI = 1 - 3*p/4
    pX = p/4
    pY = p/4
    pZ = p/4

    # Average output density matrix
    rho_avg = np.zeros((2,2), dtype=complex)

    for _ in range(shots):
        r = rng.random()

        if r < pI:
            rho_out = rho
        elif r < pI + pX:
            rho_out = X @ rho @ X
        elif r < pI + pX + pY:
            rho_out = Y @ rho @ Y
        else:
            rho_out = Z @ rho @ Z

        rho_avg += rho_out

    return rho_avg / shots

# ---- Monte Carlo demo
p = 0.37
psi = random_pure_state(seed=7)
rho = rho_from_state(psi)

A = depolarizing_A(rho, p)
rho_mc = apply_pauli_channel_monte_carlo(rho, p, shots=50000, seed=1)

print("Monte Carlo check:")
print("||A - rho_mc|| =", np.linalg.norm(A - rho_mc))
print("Should be small (goes down with more shots)")

Monte Carlo check:
||A - rho_mc|| = 0.0004993037140451906
Should be small (goes down with more shots)


In [10]:
# Quantum teleportation using |Psi-> as resource

from qiskit import QuantumCircuit, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

# --- Create circuit ---
creg = ClassicalRegister(2, 'c')
qc = QuantumCircuit(3, creg)

# Step 1: Prepare unknown state |φ> on qubit 0
qc.h(0)

# Step 2: Prepare entangled pair |Ψ-> on qubits 1 and 2
qc.h(1)
qc.cx(1, 2)
qc.z(1)
qc.x(2)

# Step 3: Bell measurement on qubits 0 and 1
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])

# Step 4: Conditional Pauli corrections on qubit 2
qc.z(2).c_if(qc.clbits[1], 1)
qc.x(2).c_if(qc.clbits[0], 1)

# Step 5: Simulate
backend = Aer.get_backend('aer_simulator')
qc.save_statevector()
job = backend.run(qc)
result = job.result()
state = result.get_statevector()
print("Final statevector:")
print(state)

# Step 6: Draw circuit
qc.draw('mpl')


CircuitError: "Circuit args must be Registers or integers. (['int', 'ClassicalRegister'] '(3, ClassicalRegister(2, 'c'))' was provided)"

In [11]:
# ===============================================================
# Quantum Teleportation using |Ψ-> = (|01> - |10>)/√2 as resource
# Works with Qiskit >= 1.0  (modular version)
# ===============================================================

from qiskit import QuantumCircuit, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

# --- Step 1: Create 3-qubit circuit with 2 classical bits ---
creg = ClassicalRegister(2, 'c')
qc = QuantumCircuit(3, creg)

# --- Step 2: Prepare the unknown state |φ> on qubit 0 ---
# Example: |φ> = (|0> + |1>)/√2  using a Hadamard
qc.h(0)

# --- Step 3: Prepare the entangled Bell pair |Ψ-> on qubits 1 and 2 ---
qc.h(1)
qc.cx(1, 2)    # creates |Φ+>
qc.z(1)        # convert |Φ+> → |Φ->
qc.x(2)        # convert |Φ-> → |Ψ->

# --- Step 4: Perform Bell measurement on qubits 0 & 1 ---
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])  # store results in classical bits c[0], c[1]

# --- Step 5: Conditional Pauli corrections on qubit 2 ---
qc.z(2).c_if(qc.clbits[1], 1)   # apply Z if c[1] = 1
qc.x(2).c_if(qc.clbits[0], 1)   # apply X if c[0] = 1

# --- Step 6: Simulate the circuit ---
backend = Aer.get_backend('aer_simulator')
qc.save_statevector()
job = backend.run(qc)
result = job.result()
state = result.get_statevector()
print("Final 3-qubit statevector:\n", state)

# --- Step 7: Draw the circuit ---
qc.draw('mpl')


CircuitError: "Circuit args must be Registers or integers. (['int', 'ClassicalRegister'] '(3, ClassicalRegister(2, 'c'))' was provided)"